In [ ]:
import json
import os
import sys

## Goal
#### In this lab we will take a step-by-step aprpoach to build an agent
* Learning Objective 1 - Build a local strands agent and run it locally
* Learning Objective 2 - Get Agent core ready for AgentCore
* Learning Objective 3 - Deploy using starter toolkit and DIY Docker image
* Learning Objective 4 - Invoke agent using toolkit, boto3, and HTTPS
* Learning Objectvie 5 - Enable tools to access protected resources - API Key and OAuth Tokens
* Learning Objectvie 6 - Use OAuth and Cognito to authenticate/authorize before accessing Agent 

<img src="images/goal.jpg" width="75%">

## Learning Objective 1 - Basic agent running locally

#### As a first step of the learning journey, we will build a simple Strands Agents
* No authentication - available to all users on the account with required IAM privileges
* get_weather() is not accessing any resources secured by an API Key or OAuth 2.0 Access Tokens
* Run locally in Jupyter Notebook
  
<img src="images/Learning 1.jpg" width="500">

In [ ]:
%%writefile weather_agent.py
import argparse, json
from strands import Agent, tool

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"

agent = Agent(tools=[weather])

def strands_agent_bedrock(payload):
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)

In [ ]:
!python3 weather_agent.py '{"prompt": "What is the weather now in Irvine?"}'

## Learning Objective 2: Get agent code ready for AgentCore

#### 1. Make code changes
Let's now deploy our agents to AgentCore Runtime. To do so we need to:

* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint decorator`
* Let AgentCoreRuntime control the running of the agent with `app.run()`

In [ ]:
%%writefile weather_agent_strands.py
import argparse, json
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"

agent = Agent(tools=[weather])

@app.entrypoint
def strands_agent_bedrock(payload):
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    app.run()

#### What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

#### 2. Create a role that Agent will use during run time
To run agent or tool in AgentCore Runtime you need an AWS Identity and Access Management execution role. The AgentCore Runtime execution role is an IAM role that AgentCore Runtime assumes to run an agent. 
Starting point for your role definition can be found at https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html. In the role definition below we have added additional actions to support identity.

In [ ]:
import boto3
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
sts = boto3.client('sts')
account_id = sts.get_caller_identity().get("Account")

In [ ]:
permission = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetWorkloadAccessTokenForJWT",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT"
            ],
            "Resource": "*"
        },
        {
            "Sid": "GetResourceOauth2Token",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceOauth2Token"
            ],
            "Resource": "*"
        },
        {
            "Sid": "GetWorkloadAccessTokenForUserId",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": "*"
        },
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "*"
        },
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer"
            ],
            "Resource": [
                "arn:aws:ecr:region:accountId:repository/*"
            ]        
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogGroup"
            ],
            "Resource": [
                "arn:aws:logs:region:accountId:log-group:/aws/bedrock-agentcore/runtimes/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogGroups"
            ],
            "Resource": [
                "arn:aws:logs:region:accountId:log-group:*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:region:accountId:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
            ]
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
        "Effect": "Allow", 
        "Action": [ 
            "xray:PutTraceSegments", 
            "xray:PutTelemetryRecords", 
            "xray:GetSamplingRules", 
            "xray:GetSamplingTargets"
            ],
         "Resource": [ "*" ] 
         },
         {
            "Effect": "Allow",
            "Resource": "*",
            "Action": "cloudwatch:PutMetricData",
            "Condition": {
                "StringEquals": {
                    "cloudwatch:namespace": "bedrock-agentcore"
                }
            }
        },
        {
            "Sid": "GetAgentAccessToken",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": [
              "arn:aws:bedrock-agentcore:region:accountId:workload-identity-directory/default",
              "arn:aws:bedrock-agentcore:region:accountId:workload-identity-directory/default/workload-identity/agentName-*"
            ]
        },
         {"Sid": "BedrockModelInvocation", 
         "Effect": "Allow", 
         "Action": [ 
                "bedrock:InvokeModel", 
                "bedrock:InvokeModelWithResponseStream"
              ], 
        "Resource": [
            "arn:aws:bedrock:*::foundation-model/*",
            "arn:aws:bedrock:region:accountId:*"
        ]
        }
    ]
}"""

trust_policy = """{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AssumeRolePolicy",
      "Effect": "Allow",
      "Principal": {
        "Service": "bedrock-agentcore.amazonaws.com"
      },
      "Action": "sts:AssumeRole",
      "Condition": {
            "StringEquals": {
                "aws:SourceAccount": "accountId"
            },
            "ArnLike": {
                "aws:SourceArn": "arn:aws:bedrock-agentcore:region:accountId:*"
            }
       }
    }
  ]
}"""
#trust_policy = json.loads(trust_policy.replace("accountId", account_id).replace("region", region))
trust_policy = trust_policy.replace("accountId", account_id).replace("region", region)
#permission = json.loads(permission.replace("accountId", account_id).replace("region", region))
permission = permission.replace("accountId", account_id).replace("region", region)

In [ ]:
agentcore_role_name = "BedrockAgentCoreRole"

In [ ]:
iam_client = boto3.client('iam')
agentcore_role_name = "BedrockAgentCoreRole"
policy_name = "BedrockAgentCorePolicy"
agentcore_iam_role = iam_client.create_role(
        RoleName=agentcore_role_name,
        AssumeRolePolicyDocument=trust_policy
    )

In [ ]:
iam_client.put_role_policy(
        PolicyDocument=permission,
        PolicyName="AgentCorePolicy",
        RoleName=agentcore_role_name
    )

## Learning Objective 3: Deploy Agent into AgentCore
You can build an Agent Docker image and deploy it on AgentCore Runtime using 2 methods
* Use Bedrock AgentCore Runtime starter toolkit to simplify the process of Docker build, ECR creation, and deployment
* Retain control of the whole process by building a Docker Image, creating an ECR, and use boto3 SDK to deploy into Amazon Bedrock AgentCore

### Option 1 - Use AgentCore Starter Toolkit

#### Step 1: Configure Runtime
During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="./images/configure.png" width="60%"/>
</div>

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
uv
boto3
bedrock-agentcore
bedrock-agentcore-starter-toolkit
tavily-python
uvicorn
fastapi

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()

agent_name = "weather_agent_strands"
config_response = agentcore_runtime.configure(
    entrypoint="weather_agent_strands.py",
    execution_role="BedrockAgentCoreRole",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

#### Step 2: Launch Agent
Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="./images/launch.png" width="75%"/>
</div>

In [ ]:
launch_response = agentcore_runtime.launch()

### Option 2 - Use Docker Container and Boto3 
AgentCore Starter toolkit abstracted the task of building a docker file and adding /invocation and /ping endpoints to your agent. However, you may have a need to build and deploy your agent in your own docker image. To align with Amazon Bedrock AgentCore requirements:
* Your agent must expose /invocations POST and /ping GET endpoints and be packaged in a Docker container
* Your docker images must be built for arm64

<div style="text-align:left">
    <img src="./images/launch_boto3.jpg" width="75%"/>
</div>

In [ ]:
agent_name = "weather_agent_strands_wo_starter"

#### Step 1 - Update agent code
This implementation:
* Creates a FastAPI application with the required endpoints
* Initializes a Strands agent for processing user messages
* Implements the /invocations POST endpoint for agent interactions
* Implements the /ping GET endpoint for health checks
* Configures the server to run on host 0.0.0.0 and port 8080

In [ ]:
%%writefile weather_agent_strands_wo_toolkit.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict, Any
from datetime import datetime
from strands import Agent, tool

app = FastAPI(title="Strands Agent Server", version="1.0.0")

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"
    
# Initialize Strands agent
strands_agent = Agent(tools=[weather])

class InvocationRequest(BaseModel):
    input: Dict[str, Any]

class InvocationResponse(BaseModel):
    output: Dict[str, Any]

    
@app.post("/invocations", response_model=InvocationResponse)
async def invoke_agent(request: InvocationRequest):
    try:
        user_message = request.input.get("prompt", "")
        if not user_message:
            raise HTTPException(
                status_code=400, 
                detail="No prompt found in input. Please provide a 'prompt' key in the input."
            )

        result = strands_agent(user_message)
        response = {
            "message": result.message,
            "timestamp": datetime.utcnow().isoformat(),
            "model": "strands-agent",
        }

        return InvocationResponse(output=response)

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Agent processing failed: {str(e)}")

@app.get("/ping")
async def ping():
    return {"status": "healthy"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8080)

#### Step 2 - Build Docker container
* Application should be launched on 0.0.0.0 and port 8080 for AgentCore Runtime to be able to invoke it.

Detailed instructions can be found at https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

In [ ]:
%%writefile Dockerfile
# Use uv's ARM64 Python base image
FROM --platform=linux/arm64 ghcr.io/astral-sh/uv:python3.11-bookworm-slim

WORKDIR /app

# Copy uv files
COPY requirements.txt .

# Install dependencies (including strands-agents)
#RUN pip install -r requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy agent file
COPY weather_agent_strands_wo_toolkit.py ./

# Expose port
EXPOSE 8080

# Run application
CMD ["uv", "run", "uvicorn", "weather_agent_strands_wo_toolkit:app", "--host", "0.0.0.0", "--port", "8080"]

In [ ]:
!docker buildx create --use

In [ ]:
!docker buildx build --quiet --platform linux/arm64 -t weather-agent:arm64 --load .

In [ ]:
!aws ecr create-repository --repository-name {agent_name} --region {region}

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.us-west-2.amazonaws.com

In [ ]:
!docker buildx build --platform linux/arm64 -t {account_id}.dkr.ecr.us-west-2.amazonaws.com/{agent_name}:latest --push .

#### Step 3 - Use Boto3 to create Agent on AgentCore Runtime

In [ ]:
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

In [ ]:
runtime_response = agentcore_control_client.create_agent_runtime(
    agentRuntimeName = agent_name,
    agentRuntimeArtifact={
        'containerConfiguration': {
            'containerUri': f'{account_id}.dkr.ecr.us-west-2.amazonaws.com/{agent_name}:latest'
        }
    },
    networkConfiguration={"networkMode": "PUBLIC"},
    roleArn=f'arn:aws:iam::{account_id}:role/{agentcore_role_name}'
)

In [ ]:
runtime_response["agentRuntimeArn"]

## Learning Objective 4: Invoke Agent
<div style="text-align:left">
    <img src="./images/invoke.png" width="60%"/>
</div>

### Option 1 - Using toolkit
You can test the Agent that you have just deployed using the 'agentcore_runtime' object. Other two options (boto3 and HTTPS endpoint) are the options that you should use for production applications
* Here we are invoking the first agent built using starter toolkit

In [ ]:
agentcore_runtime.invoke({"prompt": "How is the weather in Irvine?"})

### Option 2 - Using boto3
* Invoking the second agent deployed using custom docker and agent code

In [ ]:
agentcore_client = boto3.client("bedrock-agentcore", region_name = region)
payload = json.dumps({
    "input": {"prompt": "How is the weather in Irvine?"}
})
response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn = runtime_response["agentRuntimeArn"],
    qualifier="DEFAULT",
    payload=payload
)
response_body = response['response'].read()
response_data = json.loads(response_body)
print("Agent Response:", response_data)

#### Option 3 - Using https endpoint
* Invoking the second agent using `runtime_response["agentRuntimeArn"]` as rthe ARN.
* Can be used to invoke the first agent as well. Replace `runtime_response["agentRuntimeArn"]` with `launch_response.agent_arn`

In [ ]:
import urllib.parse, requests

# URL encode the agent ARN
escaped_agent_arn = urllib.parse.quote(runtime_response["agentRuntimeArn"], safe='')

# Construct the URL
url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
http_response = requests.post(url, data=json.dumps(payload))

In [ ]:
http_response.status_code, http_response.text

#### Why did we get a HTTP 403 (Forbidden)?
While invoking an Agent using a toolkit and boto3 SDK, we are using the IAM role of the session to invoke the agenty. However, when you invoke it as a HTTPS endpoint, there is no authorization passed to the agent. Hence the 403 Forbidden error.

## Learning Objective 5 - Tools accessing protected resources
In this learning objective, we focus on authentication within the AgentCore ecosystem. In this objective we focus on authorization within an agent so that it can access protected resources. 
The credential management system supports multiple credential types including OAuth2 access tokens, API keys, client certificates, SAML assertions, and custom authentication tokens.

#### 1 - Tool authentication using API Key
* Setup an API key credential provider for Tavily
* Get the API key from credential provider at the start of the session

##### Create credential provider
* What they do: Act as smart middlemen that connect your agents to different services and identity systems
* Built-in providers: Come ready-to-use for popular services like:
  - Google
  - GitHub  
  - Slack
  - Salesforce
* Pre-configured: These providers already have the right settings and endpoints set up for you
* Custom options: You can create your own providers for any service that uses OAuth2 authentication
* Main benefit: Saves you time by handling the complex setup work automatically

In [ ]:
import os
#os.environ["tavily_key"] = "tvly-dev-tavily-api-key"
from bedrock_agentcore.services.identity import IdentityClient
id_client = IdentityClient(region=region)
id_client.create_api_key_credential_provider({
    "name":"tavily_key_provider",
    "apiKey":os.environ["tavily_key"]
})

##### Use credential provider to get API key
* You have API keys in the AgentCore Identity vault
* Retrieve directly in your agent using the AgentCore SDK and the `@requires_api_key annotation`

In [ ]:
%%writefile weather_agent_tavily.py
import argparse, json, os
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_api_key
from tavily import TavilyClient
import asyncio

TAVILY_API_KEY_FROM_CREDS_PROVIDER = ""

@requires_api_key(
    provider_name="tavily_key_provider", # replace with your own credential provider name
    into="api_key"
)
async def need_api_key(*, api_key: str):
    global TAVILY_API_KEY_FROM_CREDS_PROVIDER
    print(f'received api key for async func: {api_key}')
    TAVILY_API_KEY_FROM_CREDS_PROVIDER = api_key

app = BedrockAgentCoreApp()


@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    response = tavily_client.search(f"What is current weather in {city}")
    return response
    
agent = Agent(tools=[weather])

@app.entrypoint
async def strands_agent_bedrock(payload):
    
    global TAVILY_API_KEY_FROM_CREDS_PROVIDER
    if not TAVILY_API_KEY_FROM_CREDS_PROVIDER:
        print("Attempting to retrieve API key...")
        try:
            await need_api_key(api_key="")
            print(f"API key retrieved: '{TAVILY_API_KEY_FROM_CREDS_PROVIDER}'")
            os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY_FROM_CREDS_PROVIDER
            print("Environment variable TAVILY_API_KEY set")
        except Exception as e:
            print(f"Error retrieving API key: {e}")
            raise
    else:
        print("API key already available")
        
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    app.run()

In [ ]:
agent_name = "weather_agent_tavily"
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
config_response = agentcore_runtime.configure(
    entrypoint="weather_agent_tavily.py",
    execution_role="BedrockAgentCoreRole",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

In [ ]:
tavily_agent_lanch = agentcore_runtime.launch()

In [ ]:
agentcore_runtime.invoke(payload={"prompt": "How is the weather in Irvine?"}, user_id=sts.get_caller_identity()["UserId"])

#### 2 - Tools needing OAuth Token - M2M Client Credentials

##### AgentCore Identity offers two ways for agents to authenticate
* User-delegated access (OAuth 2.0 authorization code grant):
  - Agent acts on behalf of a specific user
  - User gives permission for the agent to access their data
  - Good for personal assistants or user-specific tasks
* Machine-to-machine authentication (OAuth 2.0 client credentials grant):
  - Agent acts independently without a specific user
  - Direct service-to-service communication
  - Good for automated systems or background processes

In this leanring objective we will explore M2M authentication.

In [ ]:
from m2m_cognito_setup import create_m2m_pool
cognito_config = create_m2m_pool()

AgentCore Identity provice built in provider clients for Google. GitHub, Microsoft, Salesforce, and Slack. It also allows you to setup custom providers. 
In this example, we will use a custom provider that uses Amazon Cognito M2M Pool

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
        
identity_client = IdentityClient(region="us-west-2")

#Configure Cognito OAuth2 Provider - 2LO Example
cognito_provider = identity_client.create_oauth2_credential_provider({
  "name": "cognito-provider-m2m-pool-agentcore",
  "credentialProviderVendor": "CustomOauth2",
  "oauth2ProviderConfigInput": {
    "customOauth2ProviderConfig": {
      "clientId": cognito_config["client_id"],
      "clientSecret": cognito_config["client_secret"],
      "oauthDiscovery": {
        'authorizationServerMetadata': {
            'issuer': "https://cognito-idp.us-west-2.amazonaws.com/us-west-2_ZGLlSyvTI",
            'authorizationEndpoint': f"https://{cognito_config['domain_name']}.auth.us-west-2.amazoncognito.com/oauth2/authorize",
            'tokenEndpoint': f"https://{cognito_config['domain_name']}.auth.us-west-2.amazoncognito.com/oauth2/token",
            'responseTypes': ["code","token"],
        }
      }
    }
  }
})
print(cognito_provider)

In [ ]:
%%writefile weather_agent_cognito.py
import argparse, json, os
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token
from tavily import TavilyClient
import asyncio

COGNITO_ACESS_TOKEN = ""

@requires_access_token(
    provider_name="cognito-provider-m2m-pool-agentcore", # replace with your own credential provider name
    into="access_token",
    auth_flow="M2M",
    scopes = ["https://api.myapp.com/read"],
    force_authentication=True
)
async def need_access_token(*, access_token: str):
    print("")
    global COGNITO_ACESS_TOKEN
    print(f'received acess token for async func: {access_token}')
    COGNITO_ACESS_TOKEN = access_token

app = BedrockAgentCoreApp()

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    if COGNITO_ACESS_TOKEN:
        return f"Weather in {city} is warm and sunny"
    else:
        return "Cannot access weather API. Access Token missing"
    
    
agent = Agent(tools=[weather])

@app.entrypoint
async def strands_agent_bedrock(payload):
    
    global COGNITO_ACESS_TOKEN
    if not COGNITO_ACESS_TOKEN:
        print("Attempting to retrieve Token...")
        try:
            await need_access_token(access_token="")
            print(f"API key retrieved: '{COGNITO_ACESS_TOKEN}'")
            os.environ["COGNITO_ACESS_TOKEN"] = COGNITO_ACESS_TOKEN
            print("Environment variable COGNITO_ACESS_TOKEN set")
        except Exception as e:
            print(f"Error retrieving API key: {e}")
            raise
    else:
        print("Access token already available")
        
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    app.run()

In [ ]:
!rm /Users/dhegde/Documents/Code/AgentCore/.bedrock_agentcore.yaml

In [ ]:
agent_name = "weather_agent_cognito"
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
config_response = agentcore_runtime.configure(
    entrypoint="weather_agent_cognito.py",
    execution_role="BedrockAgentCoreRole",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

In [ ]:
weather_agent_cognito_launch = agentcore_runtime.launch()

In [ ]:
agentcore_runtime.invoke(payload={"prompt": "How is the weather in Irvine?"}, user_id=sts.get_caller_identity()["UserId"])

## Learning Objective 6 - Allowing only authorized users from invoking an Agent
Two authentication options for hosted agents:

IAM SigV4 Authentication (Default):
* Works automatically without setup
* Same as other AWS APIs

JWT Bearer Token Authentication (Optional):
* Requires configuration during agent creation
* Need to provide two settings:

  - **Discovery URL**: Must end with /.well-known/openid-configuration
  - **Allowed clients**: List of permitted client IDs (validates client_id claim)

* When to use each:
  - Use IAM SigV4 for standard AWS integration
  - Use JWT for external identity providers or custom auth systems

In [ ]:
#import importlib
#importlib.reload(user_cognito_setup)

#### Step 1: Create a cognito pool for user authorization

In [ ]:
import user_cognito_setup
spa_config = user_cognito_setup.create_spa_pool()

In [ ]:
!rm /Users/dhegde/Documents/Code/AgentCore/.bedrock_agentcore.yaml

#### Step 2: Add authorization configuration while creating Agent Runtime

In [ ]:
#Update the Cognito Discovery url below. You can get the discovery url from the "Provision a Cognito User Pool" section
discovery_url = f'https://cognito-idp.us-west-2.amazonaws.com/{spa_config["pool_id"]}/.well-known/openid-configuration' 
client_id = spa_config['client_id']
inbound_auth_weather_rt = Runtime()
agent_name = "inbound_auth_weather3"

response = inbound_auth_weather_rt.configure(
    entrypoint="weather_agent.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)

In [ ]:
inbound_auth_weather_launch = inbound_auth_weather_rt.launch()

In [ ]:
cognito_client = boto3.client('cognito-idp', region_name=region,)
cognito_bearer_token_1 = cognito_client.initiate_auth(
        ClientId=client_id,  
        AuthFlow="USER_PASSWORD_AUTH",
        AuthParameters={
            "USERNAME": "testuser1", 
            "PASSWORD": spa_config['user1']['password']
        }
    )["AuthenticationResult"]["AccessToken"]

In [ ]:
import urllib.parse, requests

# URL encode the agent ARN
escaped_agent_arn = urllib.parse.quote(inbound_auth_weather_launch.agent_arn, safe='')

headers = {
    "Authorization": f"Bearer {cognito_bearer_token_1}",
    "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": "aea8996f-dcf5-4227-b5ea-f9e9c1843729",
    "X-Amzn-Trace-Id": "1234567890" 
}

# Construct the URL
url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
http_response = requests.post(url, data=json.dumps({"prompt": "How is the weather in Irvine?"}), headers=headers)

In [ ]:
http_response.text